In [1]:
from IPython.display import Image
import ee
import ee.mapclient
import datetime

ee.Initialize()
L1_AGBPSeasonals = ee.ImageCollection("projects/fao-wapor/L1_AGBP")
ETaCollection = ee.ImageCollection("projects/fao-wapor/L1_AET")
AET250 = ee.ImageCollection("users/lpeiserfao/AET250")
AET_prima = ee.Image(AET250.first())

In [16]:
AET_prima.getInfo()

{u'bands': [{u'crs': u'EPSG:4326',
   u'crs_transform': [0.002232142857,
    0.0,
    -30.0044643,
    0.0,
    -0.002232142857,
    40.0044643],
   u'data_type': {u'max': 255,
    u'min': 0,
    u'precision': u'int',
    u'type': u'PixelType'},
   u'dimensions': [42564, 35844],
   u'id': u'b1'}],
 u'id': u'users/lpeiserfao/AET250/L1_AET_1501',
 u'properties': {u'area': u'AfNE',
  u'days_in_dk': 10.0,
  u'id_no': u'L1_AET_1501',
  u'level': 1.0,
  u'season': 1.0,
  u'system:asset_size': 226710072,
  u'system:footprint': {u'coordinates': [[9.335362008868378,
     40.005584280684296],
    [-1.7982352281500638, 40.00558428068428],
    [-14.41631209675748, 40.00558428068428],
    [-30.00626732411414, 40.00557958876544],
    [-30.006267324112713, -40.005579555073446],
    [-17.756391267857275, -40.005584246992285],
    [-4.3960745834483435, -40.00558424699227],
    [15.644400443173007, -40.00558424699228],
    [32.344796298689445, -40.00558424699228],
    [47.56071252260296, -40.00558424699

In [24]:
 # Actual Evapotranspiration with valid ETa values (>0 and <254)
ETaColl1 = AET250.map(lambda imm_eta: imm_eta.updateMask
                                   (imm_eta.lt(254) and (imm_eta.gt(0))))

# add image property (days in dekad) as band
ETaColl2 = ETaColl1.map(lambda imm_eta2: imm_eta2.addBands(
                                                 imm_eta2.metadata(
                                                 'days_in_dk'))) 

# get ET value, divide by 10 (as per FRAME spec) to get daily
        # value, and  multiply by number of days in dekad summed annuallyS

ETaColl3 = ETaColl2.map(lambda imm_eta3: imm_eta3.select(
            'b1').divide(10).multiply(imm_eta3.select('days_in_dk'))).sum()

In [26]:
im_uno = ee.Image(ETaColl1.first())
url_thumb_ima_uno = im_uno.getThumbUrl()
Image(url=url_thumb_ima_uno)

In [27]:
im_uno.bandNames().getInfo()

[u'b1']

In [15]:
im_due = ee.Image(ETaColl2.first())
url_thumb_ima_due = im_due.getThumbUrl()
Image(url=url_thumb_ima_due)

In [20]:
im_due.bandNames().getInfo()

[u'b1', u'days_in_dk']

In [45]:
#blank = ee.Image(100)
#maskera = blank.where(im_due.lte(0))
maskera0 = im_due.eq(0)
url_thumb_maskera0 = maskera0.getThumbUrl()
Image(url=url_thumb_maskera0)

In [47]:
maskera10 = im_due.eq(10)
url_thumb_maskera10 = maskera10.getThumbUrl()
Image(url=url_thumb_maskera10)

In [48]:
maskera11 = im_due.eq(11)
url_thumb_maskera11 = maskera11.getThumbUrl()
Image(url=url_thumb_maskera11)

In [3]:
region = [[-25.0, -37.0], [60.0, -41.0], [58.0, 39.0], [-31.0, 38.0], [-25.0, -37.0]]
scala = AET_prima.projection().nominalScale().getInfo()

visparams = {"opacity":1,
             "min": 0,
             "max": 2000,
             "palette": "d4ffc6,beffed,79c1ff,3e539f",
             "region":region,
             "scale" : scala}

In [38]:
ETaColl1_median = ETaColl1.median()
url_thumb_ETaColl1 = ETaColl1_median.getThumbUrl()
Image(url=url_thumb_ETaColl1)

In [ ]:
url_thumb_ETaColl2 = ETaColl2.getThumbUrl()
Image(url=url_thumb_ETaColl2)

In [25]:
url_thumb_ETaColl3 = ETaColl3.getThumbUrl(visparams)
Image(url=url_thumb_ETaColl3)

EEException: reduce.sum: Error in map(ID=L1_AET_1501):
Image.select: Pattern 'days_in_dk' did not match any bands.

In [ ]:
def combineImages(image, previous):
    return ee.Image(previous).addBands(ee.Image(image))
 
collection = ee.ImageCollection('MODIS/MOD13Q1').limit(2)
image = collection.iterate(combineImages, ee.Image())
print ee.Image(image).slice(1).bandNames().size().getInfo()

In [ ]:
def maskNoData(image):
            return function(image){
                    ee.Image(image.updateMask(image.lt(254) and 
                                            (image.gt(0))))            
       
illo = L1_AET_calc.iterate(maskNoData)
